### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for boto3<2.0,>=1.33.3 from https://files.pythonhosted.org/packages/09/8e/ed58e22e885332a80c31f0b80026f825e25a6c5b640e2ab01b818e490ee1/boto3-1.33.12-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00ta 0:00:01
  Obtaining dependency information for fastapi==0.95.2 from https://files.pythonhosted.org/packages/4e/1a/04887c641b67e6649bde845b9a631f73a7abfbe3afda83957e09b95d88eb/fastapi-0.95.2-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2.0.0,>=1.6.2 from https://files.pythonhosted.org/packages/e0/2f/d6f17f8385d718233bcae893d27525443d41201c938b68a4af3d591a33e4/pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 1.9 

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

##需要在线申请，然后填入自己的token
TOKEN = ''
local_model_path = Path("./bge-reranker-large")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-reranker-large"
commit_hash = "27c9168d479987529781de8474dff94d69beca11"

In [4]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

'bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11'

### 2. 把模型拷贝到S3为后续部署做准备

In [5]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
s3_model_prefix = "LLM-RAG/workshop/bge-reranker-large"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/bge-reranker-large"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/bge-reranker-large
model_snapshot_path: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11


In [7]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11/.gitattributes to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/.gitattributes
upload: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11/README.md to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/README.md
upload: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11/config.json to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/config.json
upload: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11/special_tokens_map.json to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/special_tokens_map.json
upload: bge-reranker-large/models--BAAI--bge-reranker-large/snapshots/27c9168d479987529781de8474dff94d69beca11/sentencepiece.b

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [8]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [9]:
!mkdir -p bge_reranker_large_code

In [10]:
%%writefile bge_reranker_large_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from FlagEmbedding import FlagReranker

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    model =  FlagReranker(model_location,use_fp16=True)
    
    return model

model = None

def handle(inputs: Input):
    global model 
    if not model:
        model  = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    queries = data["inputs"]
    docs = data["docs"]
    
    pairs = []
    for i,q in enumerate(queries):
        pairs.append([q,docs[i]])
    
    scores = model.compute_score(pairs)
      
    result = {"scores": scores}

    return Output().add_as_json(result)

Writing bge_reranker_large_code/model.py


In [11]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [12]:
%%writefile bge_reranker_large_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/

Writing bge_reranker_large_code/serving.properties


In [13]:
%%writefile bge_reranker_large_code/requirements.txt
transformers==4.28.1
FlagEmbedding

Writing bge_reranker_large_code/requirements.txt


In [14]:
!rm bge_reranker_model.tar.gz
!cd bge_reranker_large_code && rm -rf ".ipynb_checkpoints"
!tar czvf bge_reranker_model.tar.gz bge_reranker_large_code

rm: cannot remove 'bge_reranker_model.tar.gz': No such file or directory
bge_reranker_large_code/
bge_reranker_large_code/model.py
bge_reranker_large_code/requirements.txt
bge_reranker_large_code/serving.properties


In [15]:
s3_code_artifact = sess.upload_data("bge_reranker_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/bge-reranker-large/bge_reranker_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [16]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-reranker")# name_from_base("st-paraphrase-mpnet-base-v2") Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-reranker-2023-12-12-01-26-32-654
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:631023274615:model/bge-reranker-2023-12-12-01-26-32-654


#### 推理机型选择 (https://aws.amazon.com/cn/sagemaker/pricing/)
- GPU
  + ml.g4dn.xlarge 按需价格 0.526 USD/Hour
- CPU
  + ml.c5.xlarge   按需价格 0.204 USD/Hour

In [17]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:631023274615:endpoint-config/bge-reranker-2023-12-12-01-26-32-654-config',
 'ResponseMetadata': {'RequestId': '0755706e-738f-4c63-ada6-4234c7189751',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0755706e-738f-4c63-ada6-4234c7189751',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Tue, 12 Dec 2023 01:26:36 GMT'},
  'RetryAttempts': 0}}

In [18]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:631023274615:endpoint/bge-reranker-2023-12-12-01-26-32-654-endpoint


In [19]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Failed
Arn: arn:aws:sagemaker:us-west-2:631023274615:endpoint/bge-reranker-2023-12-12-01-26-32-654-endpoint
Status: Failed


### 5. 模型测试

In [55]:
def get_vector_by_sm_endpoint(questions, docs, sm_client, endpoint_name):
    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "docs": docs
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    scores = [item for item in json_obj['scores']]
    return scores

In [56]:
prompts1 = ["请问AWS Clean Rooms是多方都会收费吗？","请问AWS Clean Rooms是多方都会收费吗？"]
docs1 = ["会收费","生成式AI(generative AI/Gen AI)是一种AI技术,可以创造新的内容和想法的人工智能，例如图像、视频、文本、代码、音乐等。它利用机器学习模型基于大量数据进行预训练得到的超大模型也即基础模型来提供支持。"]
print(get_vector_by_sm_endpoint(prompts1, docs1, smr_client, endpoint_name))

[-1.537109375, -9.4765625]


In [40]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [41]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [42]:
!aws sagemaker delete-model --model-name {model_name}

['请问AWS Clean Rooms是多方都会收费吗？',
 '请问AWS Clean Rooms是多方都会收费吗？',
 '请问AWS Clean Rooms是多方都会收费吗？',
 '请问AWS Clean Rooms是多方都会收费吗？']